In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import pandas as pd # Dataframe 
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, normalisation, evaluation
import pylab as plt
from scipy import stats
import xlwings as xw

In [6]:
#path = "/Users/marf/Desktop/PhD Temp/2017-12-05/"
path = "/Users/marf/Desktop/PhD Temp/2016-05-19/"
#path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 4094
files_end = 4097
files_1 = range(files_start, files_end, 1)
#files_1 = range(6411,6518,1) + range(6526,6534, 1) + range(6537,6577, 1)
#files_1 = range(4094,4098,1) + range(4099,4172, 1)

In [7]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["112", "114", "115", "116", "117", "118", "119", "120", "122", "124"]
cup_config = cycles2

# Mass Range of cup configuration
mass_range = cycles2_mass_range

# Isotopes used for Interference correction
#corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Cd" : "111", "Te" : "125", "Xe" : "129"}
#corr_isotopes_2 = {}
corr_isotopes_Sb = {"Te": "126", "Xe": "129"}

norm_ratio = ["116","120"]
denom_isotope = norm_ratio[1]
Sn_monitor = ["111", "113", "125"]


def eval_iso_list(isotopes_list,norm_ratio, monitor_iso):
    isotope_nom = norm_ratio[0]
    isotope_den = norm_ratio[1]
    isotopes_list.remove(norm_ratio[1])
    for i in range(len(monitor_iso)):
        isotopes_list.append(monitor_iso[i])
    isotopes_list.sort()
    return isotopes_list

isotope_ls = eval_iso_list(Sn_isotopes, norm_ratio, Sn_monitor)

data_sample_column = [(i + "/" + denom_isotope) for i in isotope_ls]    
isotopes = [isotope_ls]

#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = True

In [8]:
database["Sn"]["Ratios"].get_all_ratios("120")

{'112': 0.029812,
 '114': 0.020195,
 '115': 0.010366,
 '116': 0.446,
 '117': 0.235313,
 '118': 0.742935,
 '119': 0.26343,
 '122': 0.142086,
 '124': 0.177588}

In [9]:

mass_range.get_graph_of_corr(corr_isotopes_2)

{'110': set(),
 '111': set(),
 '112': set(),
 '113': set(),
 '114': set(),
 '115': set(),
 '116': set(),
 '117': set(),
 '118': set(),
 '119': set(),
 '120': set(),
 '121': set(),
 '122': set(),
 '123': set(),
 '124': set(),
 '125': set(),
 '126': set(),
 '127': set(),
 '129': set(),
 '131': set()}

In [10]:
mass_range.get_mass_range()

{'110': ['Pd', 'Cd'],
 '111': ['Cd'],
 '112': ['Sn', 'Cd'],
 '113': ['Cd', 'In'],
 '114': ['Sn', 'Cd'],
 '115': ['Sn', 'In'],
 '116': ['Sn', 'Cd'],
 '117': ['Sn'],
 '118': ['Sn'],
 '119': ['Sn'],
 '120': ['Sn', 'Te'],
 '121': ['Sb'],
 '122': ['Sn', 'Te'],
 '123': ['Sb', 'Te'],
 '124': ['Sn', 'Te', 'Xe'],
 '125': ['Te'],
 '126': ['Te', 'Xe'],
 '127': ['I'],
 '129': ['Xe'],
 '131': ['Xe']}

In [11]:
mass_range.get_order_of_corr(corr_isotopes_2)

[{'110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '121',
  '122',
  '123',
  '124',
  '125',
  '126',
  '127',
  '129',
  '131'}]

In [12]:
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []

for sample in files_1:
    
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero_1["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero_1["sample"] = sample_name
    df_zero_1["cycle"] = df_zero_1.index.values
    df_zero_1.index = df_zero_1["date"]
    df_zero_1 = df_zero_1[columns_1]
    df_all = df_all.append(df_zero_1, ignore_index = True)
    
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls = blk_ls[0::5]
#blk_ls.remove(5234)
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
date,,,,,,,,,,,,,,,
2016-05-19 16:42:00,1,blank sol,0.000769,0.000527,0.005146,0.001031,0.002878,0.000892,0.002243,0.000062,0.002090,0.000846,0.001744,0.000905,0.000866
2016-05-19 16:42:00,2,blank sol,0.000842,0.000600,0.005499,0.001090,0.003130,0.000995,0.002376,0.000074,0.002060,0.000827,0.001752,0.000916,0.000849
2016-05-19 16:42:00,3,blank sol,0.000861,0.000640,0.005531,0.001127,0.003176,0.001007,0.002394,0.000068,0.001980,0.000792,0.001715,0.000880,0.000829
2016-05-19 16:42:00,4,blank sol,0.000873,0.000616,0.005674,0.001142,0.003254,0.001035,0.002499,0.000051,0.002188,0.000864,0.001880,0.000948,0.000924
2016-05-19 16:42:00,5,blank sol,0.000813,0.000600,0.005478,0.001087,0.003070,0.000990,0.002363,0.000072,0.002057,0.000809,0.001760,0.000870,0.000881
2016-05-19 16:42:00,6,blank sol,0.000730,0.000544,0.004932,0.000944,0.002710,0.000859,0.002177,0.000043,0.002069,0.000845,0.001752,0.000913,0.000877
2016-05-19 16:42:00,7,blank sol,0.000810,0.000569,0.005213,0.001069,0.002883,0.000955,0.002276,0.000080,0.002150,0.000882,0.001815,0.000923,0.000883
2016-05-19 16:42:00,8,blank sol,0.000746,0.000563,0.005067,0.001014,0.002791,0.000887,0.003720,0.000075,0.007736,0.003245,0.006397,0.003287,0.003186
2016-05-19 16:42:00,9,blank sol,0.000768,0.000541,0.004993,0.001015,0.002774,0.000908,0.002176,0.000076,0.002066,0.000827,0.001740,0.000884,0.000824


In [13]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_internal_norm = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # arbitrary blank positions
    blk1 = [item for item in blk_ls if item < sample]
    blk2 = [item for item in blk_ls if item > sample]
    blk1 = blk1[-1]
    blk2 = blk2[0]
    blk_corr_sample = True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        df_bgd_corr = new_corr.data_bgd_corr_2(df_bgd_1, df_bgd_2)
        new_corr.line2_corr(df_bgd_corr, "119")
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        new_corr.line2_corr(df_zero, "119")
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    #data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    data_sample = new_corr.internal_norm_1(norm_ratio, denom_isotope, iter_beta)
    #data_sample = new_corr.raw_ratios_corr(denom_isotope)
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = new_corr.mad_outlier_rejection(data_sample)
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample.drop(denom_isotope, axis=1)
    data_sample = data_sample[sorted(data_sample.columns)]
    data_sample.columns = data_sample_column
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_internal_norm = df_internal_norm.append(data_sample)

nu_data_reduction.py:1019: RuntimeWarning: invalid value encountered in divide
  modified_z_score = 0.6745 * diff / med_abs_deviation
nu_data_reduction.py:1021: RuntimeWarning: invalid value encountered in greater
  return modified_z_score > thresh


In [14]:
df_internal_norm= df_internal_norm.set_index("date")
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_internal_sub = pd.DataFrame(df_internal_norm, columns = columns)

df_internal_sub["Cd/Sn"] = (df_internal_norm["111"]/12.722)/(df_internal_norm["118"]/24.223)
df_internal_sub["Te/Sn"] = (df_internal_norm["125"]/7.0509)/(df_internal_norm["118"]/24.223)

#df_internal_sub["Te/Sn"] = (df_internal_norm["126"]/18.8066)/(df_internal_norm["118"]/24.223)
#df_internal_sub = df_internal_sub[df_internal_sub["118"] >= 7]
#df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")
df_internal_sub

,cycle,sample,111/120,112/120,113/120,114/120,115/120,116/120,117/120,118/120,...,116,117,118,119,120,122,124,125,Cd/Sn,Te/Sn
date,,,,,,,,,,,,,,,,,,,,,
2016-05-19 16:52:00,1,NIST 200ppb,-6.451353e-06,0.029807,-6.333982e-06,0.020181,0.010357,0.446,0.235296,0.742972,...,8.435205,4.511514,14.439805,5.190275,19.962835,2.912492,3.735608,0.000125,-1.499853e-05,0.000030
2016-05-19 16:52:00,2,NIST 200ppb,-4.975672e-06,0.029811,-2.163834e-06,0.020177,0.010359,0.446,0.235306,0.742981,...,8.843421,4.729813,15.137307,5.440462,20.924849,3.052321,3.914662,0.000144,-1.157170e-05,0.000033
2016-05-19 16:52:00,3,NIST 200ppb,-3.199865e-06,0.029814,8.696099e-07,0.020181,0.010360,0.446,0.235305,0.742971,...,8.651265,4.626971,14.807880,5.321950,20.469322,2.985970,3.829194,0.000163,-7.442444e-06,0.000038
2016-05-19 16:52:00,4,NIST 200ppb,NaN,NaN,NaN,NaN,0.010363,0.446,0.235293,0.742953,...,8.865341,4.741188,15.173768,5.453754,20.975352,3.059923,3.924385,0.000168,6.374271e-05,0.000038
2016-05-19 16:52:00,5,NIST 200ppb,NaN,NaN,NaN,NaN,0.010362,0.446,NaN,NaN,...,8.876555,4.746140,15.189300,5.459116,20.997155,3.062956,3.928068,0.000152,5.035762e-04,0.000034
2016-05-19 16:52:00,6,NIST 200ppb,NaN,NaN,NaN,NaN,0.010360,0.446,0.235306,0.742964,...,8.418274,4.502424,14.409262,5.178791,19.918907,2.905798,3.726520,0.000164,4.150429e-05,0.000039
2016-05-19 16:52:00,7,NIST 200ppb,1.063231e-05,0.029836,1.315540e-05,0.020212,0.010360,0.446,0.235295,0.742943,...,9.006123,4.816515,15.414382,5.540278,21.308089,3.108367,3.986396,0.000162,2.473199e-05,0.000036
2016-05-19 16:52:00,8,NIST 200ppb,NaN,NaN,NaN,NaN,0.010364,0.446,0.235289,0.742945,...,8.760026,4.684771,14.993219,5.388568,20.725837,3.023425,3.877250,0.000172,1.189588e-04,0.000040
2016-05-19 16:52:00,9,NIST 200ppb,2.003225e-06,0.029817,2.228576e-06,0.020189,0.010362,0.446,0.235308,0.742950,...,8.734474,4.671613,14.950404,5.373449,20.667628,3.015040,3.866807,0.000135,4.658776e-06,0.000031


In [22]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

def SD2(x):
    SD2 = 2 * np.std(x)
    return SD2

df_internal_norm_mean = df_internal_sub.drop("cycle", axis=1).groupby(df_internal_sub.index).mean()
df_internal_norm_mean_sd = df_internal_sub.drop("cycle", axis=1).groupby(df_internal_sub.index).agg([np.mean, SD2])
df_internal_norm_mean["sample"] = df_internal_sub[~df_internal_sub.index.duplicated(keep='first')]["sample"]
df_internal_norm_mean_sd["sample"] = df_internal_sub[~df_internal_sub.index.duplicated(keep='first')]["sample"]
#df_internal_norm_mean_sd = df_internal_norm_mean_sd[1:8]
#df_internal_norm_mean = df_internal_norm_mean[df_internal_norm_mean["118"] >= 7]
df_internal_norm_mean
#epsilon = epsilon.drop(["118"], axis = 1)
#plotplot = df_internal_norm_mean[(df_internal_norm_mean["125"] < 0.00010)  & (df_internal_norm_mean["124/120"] > 0.17745)] 
#plot(plotplot["125"], plotplot["124/120"], "ro")

,111/120,112/120,113/120,114/120,115/120,116/120,117/120,118/120,119/120,122/120,...,117,118,119,120,122,124,125,Cd/Sn,Te/Sn,sample
date,,,,,,,,,,,,,,,,,,,,,
2016-05-19 16:52:00,-7.588673e-07,0.029818,0.000001,0.020187,0.010361,0.446,0.235305,0.742963,0.263472,0.142097,...,4.620872,14.788473,5.31522,20.443434,2.982275,3.824765,0.000156,0.000039,0.000036,NIST 200ppb


In [122]:
df_internal_norm_sample_sd = df_internal_norm_mean.groupby(df_internal_norm_mean["sample"]).agg([np.mean, SD2])
df_internal_norm_sample_sd

111/120        112/120        113/120        114/120       \
                     mean  SD2      mean  SD2      mean  SD2      mean  SD2   
sample                                                                        
NIST 200ppb -7.588673e-07  0.0  0.029818  0.0  0.000001  0.0  0.020187  0.0   

              115/120      ...        122            124            125       \
                 mean  SD2 ...       mean  SD2      mean  SD2      mean  SD2   
sample                     ...                                                 
NIST 200ppb  0.010361  0.0 ...   2.982275  0.0  3.824765  0.0  0.000156  0.0   

                Cd/Sn          Te/Sn       
                 mean  SD2      mean  SD2  
sample                                     
NIST 200ppb  0.000039  0.0  0.000036  0.0  

[1 rows x 54 columns]

In [123]:
#df_internal_norm_mean.describe()

In [ ]:
# variation of standard data + mean + 2SD
sel_ratio = "124/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].mean(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].mean()+ 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [ ]:
# variation of standard data + meadian + 25% quantile + 75% quantile
sel_ratio = "120/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].median(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].quantile(0.25), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].quantile(0.75), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [ ]:
# outlier corr variation of standard data + meadian + 25% quantile + 75% quantile
sel_ratio = "120/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = new_corr.mad_outlier_rejection(plot_Nist.drop("sample", axis=1))
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].mean(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].mean()+ 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [47]:
std_used = "NIST 200ppb"
sub_index = df_internal_norm_mean[(df_internal_norm_mean["sample"].shift(1) == df_internal_norm_mean["sample"].shift(-1)) & (df_internal_norm_mean["sample"].shift(1) == std_used)].index
#corr_values = df_internal_norm_mean[(df_external_norm_mean >= 0.74289)]
#corr_values = df_internal_norm_mean[:"20150110 10:00"]
corr_values = df_internal_norm_mean[df_internal_norm_mean["118"] >= 0.1]
corr_values_2 = corr_values[corr_values["sample"] == std_used]
#corr_values = df_internal_norm_mean
sample = corr_values["sample"]
sample_2 = corr_values_2["sample"]
std_mean = corr_values[(corr_values["sample"] == std_used)].mean()
new = corr_values[corr_values.columns[corr_values.dtypes != 'object']]
new_2 = corr_values_2[corr_values_2.columns[corr_values_2.dtypes != 'object']]

#std_mean["112/120", "114/120", "115/120", "117/120", "118/120", "119/120", "122/120", "124/120"] = [0.029823, 0.020189, 0.010372, 0.235299, 0.742923, 0.263443, 0.142078, 0.177546]
#print std_mean
#epsilon =  ((new / std_mean)-1)*10000
#new["sample"] = sample
epsilon_NIST =  ((new / ((new.shift(1) + new.shift(-1))/2))-1)*10000
epsilon_2 = ((new_2 / ((new_2.shift(1) + new_2.shift(-1))/2))-1)*10000
epsilon_NIST["sample"] = sample
epsilon_2["sample"] = sample

epsilon_NIST = epsilon_NIST[new.index.isin(sub_index)]
epsilon_NIST= epsilon_NIST[epsilon_NIST["sample"] != std_used]
epsilon_2 = epsilon_2[epsilon_2["sample"] == std_used]
epsilon_comp = epsilon_NIST.append(epsilon_2, ignore_index=False)
#epsilon_comp.index = epsilon_comp["Date"]
#epsilon_comp.columns
epsilon_comp = epsilon_comp.sort_index(axis=0)

In [48]:
#epsilon_comp = epsilon_comp.drop([datetime.strptime("2017-02-24 15:18", "%Y-%m-%d %H:%M")])
epsilon_comp

,111/118,112/118,113/118,114/118,115/118,116/118,117/118,119/118,120/118,122/118,...,117,118,119,120,122,124,125,Cd/Sn,Te/Sn,sample
date,,,,,,,,,,,,,,,,,,,,,
2016-05-19 16:52:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIST 200ppb
2016-05-19 17:16:00,-96044.226939,-0.387860,24546.451890,-0.294964,-1.008319,-0.071257,-0.072493,-0.109872,-0.180770,-4.440892e-12,...,8267.027952,8265.863270,8264.383344,8262.955792,8260.775620,8258.471258,6837.708404,-95951.866464,-790.272144,NIST 350ppb
2016-05-19 17:49:00,-7240.797528,-0.457734,-10588.527913,-0.632712,-0.134480,-0.047296,0.114229,0.070394,-0.039179,-1.110223e-12,...,-297.649333,-297.183667,-296.546579,-296.106823,-294.933310,-293.915995,111.363312,-7238.968771,414.743441,NIST 200ppb
2016-05-19 18:13:00,21871.016975,0.956515,21003.587308,1.173780,0.057999,0.074482,-0.158342,-0.144034,0.061798,2.220446e-12,...,129.115982,129.644163,129.865699,130.439631,131.093575,131.957906,-23.223319,21861.621099,-148.760936,NIST 200ppb
2016-05-19 18:27:00,-6278.231287,-0.626537,-4097.624563,-0.625306,-0.193762,-0.027907,0.058031,0.104988,0.013186,-4.440892e-12,...,56.257717,55.910832,55.727773,55.350989,54.776243,54.157674,41.183087,-6892.933677,-21.803724,NIST 200ppb
2016-05-19 18:41:00,-154438.670913,0.387453,44341.941973,1.046339,-0.167618,-0.094748,-0.069347,-0.078124,-0.084475,4.440892e-12,...,-112.259557,-112.362021,-112.609844,-112.784689,-113.031947,-113.254837,-79.408742,-199543.927005,54.944619,NIST 200ppb
2016-05-19 18:55:00,-2489.482894,-0.542017,-2746.782910,-0.724580,-0.230257,0.069401,-0.029631,0.044379,-0.212600,-3.330669e-12,...,8506.183722,8506.217963,8506.190122,8505.610144,8505.799689,8505.631939,9074.727021,-5133.996965,298.712933,NIST 350ppb
2016-05-19 19:09:00,-14329.023216,-0.217091,-11547.909041,-1.159715,0.187001,0.120487,0.117703,0.060185,0.119977,-3.330669e-12,...,48.659129,48.565517,48.654002,48.740633,48.666490,48.702752,-276.650763,-13744.037168,-341.007636,NIST 200ppb
2016-05-19 19:33:00,115773.288395,0.175579,27522.664845,0.741142,-0.090564,-0.067249,-0.157023,-0.079115,-0.147791,0.000000e+00,...,-45.763607,-45.166469,-44.826863,-44.444565,-43.436993,-42.656238,248.897372,115838.564152,303.055529,NIST 200ppb


In [49]:
def SD2(x):
    SD2 = np.nanstd(x, ddof=1) * 2
    return SD2

epsilon = epsilon_comp.drop(["111", "125", "112", "114", "115", "116", "117", "118", "119", "120", "122", "124", "Cd/Sn", "Te/Sn"], axis = 1)
#epsilon = epsilon.drop(["118"], axis = 1)
epsilon_sum = np.round(epsilon.groupby("sample").agg([np.mean, SD2]), decimals=2)
epsilon_sum

/Users/marf/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


111/118           112/118         113/118  \
                              mean       SD2    mean   SD2      mean   
sample                                                                 
Allende HP 1_1            26745.04       NaN   -0.09   NaN  20238.99   
Allende bomb 1_1          50674.39       NaN    0.19   NaN  62481.27   
NIST + Cd ns 1             6009.59       NaN   -0.60   NaN   2371.41   
NIST 200ppb                2009.75  69241.41   -0.00  1.10   4328.69   
NIST 200ppb + 0.005ppb U  -4535.86   9714.25    0.28  0.15  -3613.81   
NIST 200ppb + 0.02ppb Mo  16576.79   4160.71   -0.02  0.43  10310.76   
NIST 200ppb + 0.03ppb Cd  94857.89  99805.52    0.07  0.17  87862.50   
NIST 200ppb + 0.6ppb Pd   -2515.66   3363.00    0.09  0.12  -2094.64   
NIST 350ppb              -20633.85  84538.41   -0.25  0.86   3026.89   
ZH09-05 23.3_1_3           8280.75       NaN    0.69   NaN  11417.05   
ZH09-05 23.3_1_4          19481.64       NaN   -0.06   NaN  13850.34   
ZH09-05 44.8_1_2          -2286.83       NaN    0.07   NaN  -3149.64   

                                   114/118       115/118          ...     \
                               SD2    mean   SD2    mean   SD2    ...      
sample                                                            ...      
Allende HP 1_1                 NaN    0.05   NaN   -0.98   NaN    ...      
Allende bomb 1_1               NaN    1.72   NaN   -2.89   NaN    ...      
NIST + Cd ns 1                 NaN   -0.66   NaN   -1.01   NaN    ...      
NIST 200ppb               29282.73    0.01  1.75   -0.01  1.05    ...      
NIST 200ppb + 0.005ppb U   8950.95    0.39  0.82   -0.91  0.11    ...      
NIST 200ppb + 0.02ppb Mo  19022.38   -0.32  0.70    0.95  0.13    ...      
NIST 200ppb + 0.03ppb Cd  88724.47    0.45  0.76   -0.69  1.45    ...      
NIST 200ppb + 0.6ppb Pd    2183.80   -0.11  0.02    0.41  0.68    ...      
NIST 350ppb               24435.34   -0.31  1.85   -0.12  1.36    ...      
ZH09-05 23.3_1_3               NaN    0.79   NaN   -1.34   NaN    ...      
ZH09-05 23.3_1_4               NaN   -0.06   NaN   -1.77   NaN    ...      
ZH09-05 44.8_1_2               NaN   -0.06   NaN   -1.26   NaN    ...      

                         120/118       122/118      124/118        125/118  \
                            mean   SD2    mean  SD2    mean   SD2     mean   
sample                                                                       
Allende HP 1_1             -0.14   NaN     0.0  NaN   -0.20   NaN -1466.98   
Allende bomb 1_1            0.02   NaN     0.0  NaN   -0.00   NaN  -170.18   
NIST + Cd ns 1              0.06   NaN     0.0  NaN    0.09   NaN  -261.76   
NIST 200ppb                 0.00  0.12    -0.0  0.0    0.00  0.25    30.84   
NIST 200ppb + 0.005ppb U    0.00  0.02     0.0  0.0    0.07  0.25 -2747.92   
NIST 200ppb + 0.02ppb Mo   -0.07  0.02     0.0  0.0   -0.06  0.15  -100.68   
NIST 200ppb + 0.03ppb Cd   -0.04  0.05     0.0  0.0   -0.04  0.22    87.31   
NIST 200ppb + 0.6ppb Pd    -0.01  0.05    -0.0  0.0    0.00  0.13   605.78   
NIST 350ppb                -0.26  0.13    -0.0  0.0    0.05  0.15  -417.33   
ZH09-05 23.3_1_3            0.09   NaN     0.0  NaN    0.17   NaN  -155.52   
ZH09-05 23.3_1_4           -0.02   NaN    -0.0  NaN   -0.02   NaN -1260.09   
ZH09-05 44.8_1_2            0.08   NaN     0.0  NaN    0.06   NaN  -160.18   

                                        113            
                              SD2      mean       SD2  
sample                                                 
Allende HP 1_1                NaN  21912.54       NaN  
Allende bomb 1_1              NaN  63720.87       NaN  
NIST + Cd ns 1                NaN   2976.40       NaN  
NIST 200ppb               1471.00   4826.38  31460.84  
NIST 200ppb + 0.005ppb U  5878.89  -3640.10   8965.32  
NIST 200ppb + 0.02ppb Mo   813.04  10037.17  18600.04  
NIST 200ppb + 0.03ppb Cd   246.88  90050.04  88675.04  
NIST 200ppb + 0.6ppb Pd     24.26  -1841.31   3268.54  
NIST 350ppb       

In [ ]:
### sel_ratio = "112/118"
std = "NIST 100ppb"

def plot_epsilon(sel_ratio, std, xlim):
    x = range(1, len(epsilon_sum.index)+1)
    plt.errorbar(epsilon_sum[sel_ratio]["mean"], x, xerr = epsilon_sum[sel_ratio]["SD2"], ls='none', marker="o")
    plt.title("samples" + ", 2SD = " + str(np.round((2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std()),2)) + ", n= " +  str((epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].count())))
    plt.xlim(xlim)
    plt.ylim([0.5, len(epsilon_sum.index)+1])
    plt.yticks(x, epsilon_sum.index, size='small', rotation=0)
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean(), color='k', linestyle='-')
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean()+ 2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std(), color='r', linestyle='--')
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean()- 2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std(), color='r', linestyle='--')
    #plt.axvline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
    plt.ylabel(r'$\mathregular{\epsilon^{'+sel_ratio+'}}$Sn$_{NIST}$')
    #savefig(path+"epsilon_118.pdf", bbox_inches='tight')

In [ ]:
wb = xw.books.add()
#wb.sheets.add()
wb.sheets["Sheet1"].range('A1').value = df_all
df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).mean()
df_all_mean["sample"] = df_all[~df_all.index.duplicated(keep='first')]["sample"]
wb.sheets["Sheet1"].range('A344').value = df_all_mean

In [ ]:
#wb.sheets.add()
wb.sheets[3].range('AF1').value = df_internal_sub
#df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).mean()
#df_all_mean["sample"] = df_all[~df_all.index.duplicated(keep='first')]["sample"]
#xw.Range('A344').value = df_all_mean

In [ ]:
#wb.sheets.add()
wb.sheets[4].range('A21').value = df_internal_norm_mean_sd

In [ ]:
wb.sheets[4].range('BF21').value = epsilon_comp

In [ ]:
#epsilon_comp[epsilon_comp["sample"]== "NIST 200ppb"]["117/118"].std()
wb.sheets[4].range('BF36').value = epsilon_sum

In [ ]:
#sel_ratio = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
#sel_ratio = ["116/117", "118/117", "119/117", "120/117", "122/117", "124/117"]
sel_ratio = ["116/120", "117/120", "118/120", "119/120", "122/120", "124/120"]
#sel_ratio = ["112/120", "114/120", "115/120"]
#sel_ratio = ["112/118", "114/118", "115/118"]
xlim = [-1, 1]

fig = plt.figure(figsize=(10,15))

for i in range(len(sel_ratio)):
    ax = plt.subplot(5,2,i+1)
    plot_epsilon(sel_ratio[i], "NIST 200ppb", xlim)
fig.tight_layout()
#wb.sheets[4].pictures.add(fig, name= '116/120 norm - bgd_corr 1', update = True)
#savefig(path+"epsilon_x_bgd_118_2_100ppb.pdf", bbox_inches='tight')

In [ ]:
sel_ratio = "116/118"
epsilon_comp = epsilon_comp[epsilon_comp["sample"] == "NIST 200ppb"]
x = epsilon_comp.index
plt.plot(x, epsilon_comp[sel_ratio], marker="o")
plt.ylim([-0.5, 0.5])
#plt.xticks(x, epsilon_comp["sample"], size='small', rotation=90)
plt.axhline(epsilon_comp[sel_ratio].mean(), color='k', linestyle='-')
plt.axhline(epsilon_comp[sel_ratio].mean()+ 2*epsilon_comp[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(epsilon_comp[sel_ratio].mean()- 2*epsilon_comp[sel_ratio].std(), color='r', linestyle='--')
plt.ylabel(r'$\mathregular{\epsilon^{118}}$Sn')
#savefig(path+"epsilon_119.pdf", bbox_inches='tight')

In [76]:
df_all.to_csv(path + "Sn_H8_L4_2_cycles_NU_Signals_zero_corr.csv", mode='w', header=True)

In [89]:
df_internal_norm.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_122_118_bgd_corr_outlier_corr_all_test.csv", mode='w', header=True)

In [60]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_122_118_bgd_corr_outlier_corr_den_corr.csv", mode='w', header=True)

In [ ]:
df_std.to_csv(path_sav + "All_standards_internal_norm_mean.csv")

In [ ]:
df_all_mean_1 = pd.read_csv(path_sav + "Sn_H9_L4_1_cycles_internal_norm_mean.csv")

In [ ]:
df_all_mean_2 = pd.read_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm_mean.csv")

In [ ]:
df_mean_all = df_all_mean_1.append(df_all_mean_2)

In [ ]:
#old Cd/Sn & Te/Sn extraction - doesn't include bgd corr
df_internal_norm= df_internal_norm.set_index("date")
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_internal_sub = pd.DataFrame(df_internal_norm, columns = columns)

df_all_sub = df_all[(df_all["sample"] != "blank sol") & (df_all["sample"] != "SQ") & (df_all["sample"] != "teflon blank")]
df_internal_sub["111"] = df_all_sub["L3 (1)"]
df_internal_sub["125"] = df_all_sub["H5 (2)"]
df_internal_sub["Cd/Sn"] = (df_all_sub["L3 (1)"]/12.722)/(df_all_sub["H4 (1)"]/24.223)
df_internal_sub["Te/Sn"] = (df_all_sub["H5 (2)"]/7.0509)/(df_all_sub["H4 (1)"]/24.223)
#df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")

In [ ]:
df_mean_all = df_mean_all[2:]
df_mean_all.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
df_mean_all = df_mean_all[["Date", "112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118","sample"]]
df_mean_all = df_mean_all.set_index("Date")

In [ ]:
df_std = df_mean_all[(df_mean_all["sample"] == "SPEX1 100ppb") | (df_mean_all["sample"] == "SPEX 100ppb") | (df_mean_all["sample"] == "NIST 100ppb")]
df_std

In [ ]:
df_std[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118"]] = df_std[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118"]].astype(float)
df_std.index = df_std.index.astype(datetime64)
df_std = df_std[(df_std.index < pd.Timestamp('2015-03-10')) | (df_std.index > pd.Timestamp('2015-03-12'))]
df_std = df_std[(df_std.index < pd.Timestamp('2014-11-29 18:25:00')) | (df_std.index > pd.Timestamp('2014-11-29 20:27:00'))]
df_std

In [ ]:
df_std = df_std[(df_std["118"] > 5) & (df_std["118/120"] > 0.7428)]
for i in df_std.index[df_std["sample"] == "SPEX1 100ppb"]:
    df_std.ix[i, "sample"] = "SPEX 100ppb"

In [ ]:
sample = df_std["sample"]
std_mean = df_std[(df_std["sample"] == "SPEX 100ppb")].mean()
new = df_std[df_std.columns[df_std.dtypes != 'object']]
epsilon =  ((new / std_mean)-1)*10000
epsilon["sample"] = sample
epsilon
epsilon[(epsilon["118/120"] > 0.4) | (epsilon["118/120"] < -0.4)]

In [ ]:
lee = pd.DataFrame({"112/120" : pd.Series(0.029812), "114/120" : pd.Series(0.020195), "115/120": pd.Series(0.010366), "116/120": pd.Series(0.4460), "117/120" : pd.Series(0.235313), "118/120" : pd.Series(0.742935), "119/120" : pd.Series(0.263430), "122/120" : pd.Series(0.142086), "124/120" : pd.Series(0.177588)})
epsilon_lee = ((lee / std_mean[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120"]])-1)*10000
epsilon_lee["sample"] = "Lee et al. (1995)"
epsilon_lee